In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using PyPlot
using ForwardDiff

In [ ]:
ℓ = 1.0
m = 1.0
c = 30.0
g = 9.81

function f(x)
    #pendulum dynamics
    
    θ = x[1]
    θ̇ = x[2]
    
    ẋ = [θ̇; -(g/ℓ)*sin(θ)-c*θ̇]
end

In [ ]:
function euler_step(xk)
    xn = xk + h*f(xk)
end

In [ ]:
function taylor_step(xk)
    Ak = ForwardDiff.jacobian(f,xk)
    xn = xk + h*f(xk) + 0.5*h*h*Ak*f(xk)
end

In [ ]:
function implicit_midpoint_step(xk)
    xn = xk
    r = xn - xk - h*f(0.5*xk + 0.5*xn)
    while maximum(abs.(r)) > 1e-12
        R = ForwardDiff.jacobian(x -> x-xk-h*f(0.5*xk + 0.5*x), xn)
        Δxn = -R\r
        xn += Δxn
        r = xn - xk - h*f(0.5*xk + 0.5*xn)
    end
    
    return xn
end

In [ ]:
function explicit_midpoint_step(xk)
    xm = xk + 0.5*h*f(xk)
    xn = xk + h*f(xm)
end

In [ ]:
function simulate_euler!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= euler_step(xtraj[:,k])
    end
end

In [ ]:
function simulate_taylor!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= taylor_step(xtraj[:,k])
    end
end

In [ ]:
function simulate_implicit_mid!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= implicit_midpoint_step(xtraj[:,k])
    end
end

In [ ]:
function simulate_explicit_mid!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= explicit_midpoint_step(xtraj[:,k])
    end
end

In [ ]:
Tf = 10.0
h = 0.1 #50 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [ ]:
x0 = [30*(pi/180); 0.0]
xtraj1 = zeros(2,N)
xtraj2 = zeros(2,N)
xtraj3 = zeros(2,N)
xtraj4 = zeros(2,N)
xtraj1[:,1] = x0;
xtraj2[:,1] = x0;
xtraj3[:,1] = x0;
xtraj4[:,1] = x0;

In [ ]:
simulate_euler!(xtraj1, N)

In [ ]:
simulate_taylor!(xtraj2, N)

In [ ]:
simulate_implicit_mid!(xtraj3, N)

In [ ]:
simulate_explicit_mid!(xtraj4, N)

In [ ]:
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo

vis = Visualizer()
TrajOptPlots.set_mesh!(vis, RobotZoo.Pendulum())
render(vis)

In [ ]:
X1 = [SVector{2}(x) for x in eachcol(xtraj3)];
visualize!(vis, RobotZoo.Pendulum(), thist[end], X1)

In [ ]:
#plot(thist,xtraj1[1,:])
#plot(thist,xtraj2[1,:])
plot(thist,xtraj3[1,:])
plot(thist,xtraj4[1,:])
xlabel("Time (sec)")
ylabel("θ (rad)")

In [ ]:
#Let's plot the total energy of the system
Ehist1 = 0.5*m*ℓ^2*(xtraj1[2,:].^2) .+ m*g*ℓ*(1.0.-cos.(xtraj1[1,:]))
Ehist2 = 0.5*m*ℓ^2*(xtraj2[2,:].^2) .+ m*g*ℓ*(1.0.-cos.(xtraj2[1,:]))
Ehist3 = 0.5*m*ℓ^2*(xtraj3[2,:].^2) .+ m*g*ℓ*(1.0.-cos.(xtraj3[1,:]))
Ehist4 = 0.5*m*ℓ^2*(xtraj4[2,:].^2) .+ m*g*ℓ*(1.0.-cos.(xtraj4[1,:]))
#plot(thist,Ehist1)
#plot(thist,Ehist2)
plot(thist,Ehist3)
#plot(thist,Ehist4)
xlabel("Time (sec)")
ylabel("Total Energy")